In [1]:
import cv2,os
import numpy as np
# import random
# import matplotlib.pyplot as plt
# import pandas as pd
import pywt
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
def process_frame(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    adaptive_thresh = cv2.adaptiveThreshold(blurred_frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV, 11, 2)
    edges = cv2.Canny(adaptive_thresh, 50, 150)
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours, gray_frame

# Function to process frames for focus measures
def process_gray_frame(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    resized_frame = cv2.resize(blurred_frame, (256, 256))
    return resized_frame

# Define focus measure functions
def laplacian_of_gaussian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def tenengrad(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    return np.mean(gradient_magnitude)

def brenner(image):
    shifted_image = np.roll(image, -2, axis=1)
    return np.sum((image[:, :-2] - shifted_image[:, :-2])**2)

def gray_level_variance(image):
    return np.var(image)

def entropy(image):
    histogram = cv2.calcHist([image], [0], None, [256], [0, 256])
    histogram = histogram / histogram.sum()
    entropy_value = -np.sum(histogram * np.log2(histogram + 1e-10))
    return entropy_value

def wavelet_focus_measure(image):
    coeffs = pywt.wavedec2(image, 'db1', level=4)
    return sum(np.sum(np.abs(coeff)) for coeff in coeffs[1:])

def max_absolute_gradient(image):
    sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return np.max(np.abs(sobel_x) + np.abs(sobel_y))

focus_functions = [
    laplacian_of_gaussian,
    # tenengrad, brenner,
    # gray_level_variance,
    # entropy, wavelet_focus_measure,
    # max_absolute_gradient
]

# Segment image into smaller grids
def segment_image(image, grid_size):
    segments = []
    h, w = image.shape
    step_h, step_w = h // grid_size[0], w // grid_size[1]
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            segment = image[i*step_h:(i+1)*step_h, j*step_w:(j+1)*step_w]
            segments.append(segment)
    return segments

# Extract local features from segmented image
def extract_local_features(image, grid_size, focus_functions):
    segments = segment_image(image, grid_size)
    features = []
    for segment in segments:
        features.extend([func(segment) for func in focus_functions])
    return features

In [ ]:
experiment_image_path = "./xxx/Experiment_Image"
rendering_image_path = "./xxx/Physical_Rendering_Image/Output"
kinematic_path = "./xxx/Kinematics_Info"
experiment_image_counts = {}
rendering_image_counts = {}

for folder_name in os.listdir(experiment_image_path):
    folder_path = os.path.join(experiment_image_path, folder_name)
    if os.path.isdir(folder_path): 
        image_count = sum(1 for file in os.listdir(folder_path)
                          if file.lower().endswith(('.png', '.jpg', '.jpeg')))
        experiment_image_counts[folder_name] = image_count

for folder_name in os.listdir(rendering_image_path):
    folder_path = os.path.join(rendering_image_path, folder_name)
    if os.path.isdir(folder_path): 
        image_count = sum(1 for file in os.listdir(folder_path)
                          if file.lower().endswith(('.png', '.jpg', '.jpeg')))
        rendering_image_counts[folder_name] = image_count

experiment_min_count = min(experiment_image_counts.values())
rendering_min_count = min(rendering_image_counts.values())
print("min experiment count: ", experiment_min_count)
print("min rendering count: ", rendering_min_count)

In [ ]:
from torchvision import transforms
from scipy.signal import find_peaks
import numpy as np
from scipy.signal import find_peaks
grid_size=(1, 1)
center_crop = transforms.CenterCrop(300)

# Output directory for the paired images
output_path = "./xxx/Pix2Pix_All"
os.makedirs(output_path, exist_ok=True)

def format_folder_name(folder):
    parts = folder.split("_")
    formatted = f"P{int(parts[0][1:]):02d}_R{int(parts[1][1:]):02d}"
    return formatted

def save_list_to_txt(filename, data):
    with open(filename, 'w') as f:
        for item in data:
            f.write(f"{item}\n")

for folder_name in os.listdir(experiment_image_path):
############################### Process Experiment Data ############################### 
    print("experiment_image_path", folder_name)
    kinematic_file_path = os.path.join(kinematic_path, folder_name + '.txt')
    kinematic_values = np.loadtxt(kinematic_file_path)[1:-1]

    experiment_folder_path = os.path.join(experiment_image_path, folder_name)
    experiment_focus_features = []
    for experiment_file_name in os.listdir(experiment_folder_path):
        experiment_file_path = os.path.join(experiment_folder_path, experiment_file_name)
        
        # Read the image
        experiment_image = Image.open(experiment_file_path)
        experiment_image_np = np.array(experiment_image)
        # print(image_np.shape)

        experiment_gray_cropped_frame = process_gray_frame(experiment_image_np)

        experiment_features = extract_local_features(experiment_gray_cropped_frame, grid_size, focus_functions)
        experiment_focus_features.append(experiment_features)

    # Combnine experiment_focus_features with depth value (kinematic data)
    if len(kinematic_values) != len(experiment_focus_features):
        print(f"Error: Length mismatch! kinematic_values has {len(kinematic_values)} elements, but experiment_focus_features has {len(experiment_focus_features)} elements.")
        break
    else:
        experiment_focus_features = np.column_stack((experiment_focus_features, kinematic_values))
        # print("Data combined successfully.")

    experiment_data = np.array(experiment_focus_features)
    experiment_data_min = experiment_data.min(axis=0)  # Minimum of each column
    experiment_data_max = experiment_data.max(axis=0)  # Maximum of each column
    experiment_normalized_data = (experiment_data - experiment_data_min) / (experiment_data_max - experiment_data_min)

    # experiment_peaks, _ = find_peaks(experiment_normalized_data[:500,0], height=0.6, distance=10)
    # experiment_peak_points = [experiment_normalized_data[i,0] for i in experiment_peaks]

    # # Plot the first column
    # plt.figure(figsize=(10, 5))
    # plt.plot(experiment_normalized_data[:500,0], label='experiment-0')
    # plt.plot(experiment_normalized_data[:500,1], label='kinematic')
    # plt.scatter(experiment_peaks, experiment_peak_points, color='r', zorder=5, label="Peak Points")
    # plt.ylabel('Value')
    # plt.legend(loc='upper left', bbox_to_anchor=(-0.2, 1))
    # plt.title('Plot of the First Column')
    # plt.show()

############################### Process Rendering Data ############################### 
    rendering_folder_name = format_folder_name(folder_name)
    print("rendering_image_path", rendering_folder_name)
    rendering_folder_path = os.path.join(rendering_image_path, rendering_folder_name)
    rendering_focus_features = []
    for rendering_file_name in os.listdir(rendering_folder_path):
        rendering_file_path = os.path.join(rendering_folder_path, rendering_file_name)
        
        # Read the image
        rendering_image = Image.open(rendering_file_path)
        rendering_image = center_crop(rendering_image)
        rendering_image_np = np.array(rendering_image)
        # print(image_np.shape)

        rendering_blurred_frame = cv2.GaussianBlur(rendering_image_np, (5, 5), 0)
        rendering_resized_frame = cv2.resize(rendering_blurred_frame, (256, 256))

        rendering_features = extract_local_features(rendering_resized_frame, grid_size, focus_functions)
        rendering_focus_features.append(rendering_features)

        # plt.figure(figsize=(5, 5))
        # plt.imshow(resized_frame, cmap="gray")
        # plt.axis('off')  # Hide axes
        # plt.title(f"{folder_name} - {file_name}")
        # plt.show()

    rendering_data = np.array(rendering_focus_features)
    rendering_data_min = rendering_data.min(axis=0)  # Minimum of each column
    rendering_data_max = rendering_data.max(axis=0)  # Maximum of each column
    rendering_normalized_data = (rendering_data - rendering_data_min) / (rendering_data_max - rendering_data_min)


############################### Data Alignment ############################### 
    rendering_peak_indices = [0] + [i - 1 for i in range(40, 601, 40)]
    rendering_peak_points = [rendering_normalized_data[i] for i in rendering_peak_indices]

    experiment_peak_indices, _ = find_peaks(experiment_normalized_data[:,0], height=0.6, distance=10)
    experiment_peak_points = [experiment_normalized_data[i,0] for i in experiment_peak_indices]

    aligned_rendering_data = []
    aligned_experiment_data = []
    aligned_experiment_kinetic = []
    aligned_rendering_indices = []
    aligned_experiment_indices = []

    for i in range(len(rendering_peak_indices) - 1):
        rendering_start, rendering_end = rendering_peak_indices[i], rendering_peak_indices[i + 1]
        experiment_start, experiment_end = experiment_peak_indices[i], experiment_peak_indices[i + 1]

        # Number of points in each segment
        m = rendering_end - rendering_start
        n = experiment_end - experiment_start
        # print(i, ": m - ",m, ", n - ",n)

        if m < n:
            sampled_experiment_indices = np.sort(np.random.choice(range(experiment_start, experiment_end), m, replace=False))
            sampled_experiment_points = [experiment_normalized_data[idx, 0] for idx in sampled_experiment_indices]
            sampled_experiment_kinetic = [experiment_normalized_data[idx, 1] for idx in sampled_experiment_indices]
            rendering_points = rendering_normalized_data[rendering_start:rendering_end, 0]
            rendering_indices = list(range(rendering_start, rendering_end))
            # selected_experiment_points = np.random.choice(experiment_normalized_data[experiment_peak_indices[i]:experiment_peak_indices[i + 1], 0], m, replace=False).tolist()
            # selected_rendering_points = rendering_normalized_data[rendering_peak_indices[i]:rendering_peak_indices[i + 1], 0]
        else:
            sampled_rendering_indices = np.sort(np.random.choice(range(rendering_start, rendering_end), n, replace=False))
            sampled_rendering_points = [rendering_normalized_data[idx, 0] for idx in sampled_rendering_indices]
            sampled_experiment_points = experiment_normalized_data[experiment_start:experiment_end, 0]
            sampled_experiment_indices = list(range(experiment_start, experiment_end))
            sampled_experiment_kinetic = experiment_normalized_data[experiment_start:experiment_end, 1]
            rendering_points = sampled_rendering_points
            rendering_indices = sampled_rendering_indices

            # selected_rendering_points = np.random.choice(rendering_normalized_data[rendering_peak_indices[i]:rendering_peak_indices[i + 1], 0], n, replace=False).tolist()
            # selected_experiment_points = experiment_normalized_data[experiment_peak_indices[i]:experiment_peak_indices[i + 1], 0]

        aligned_rendering_data.append(rendering_points)
        aligned_experiment_data.append(sampled_experiment_points)
        aligned_experiment_kinetic.append(sampled_experiment_kinetic)
        aligned_rendering_indices.append(rendering_indices)
        aligned_experiment_indices.append(sampled_experiment_indices)

    # Combine aligned data for visualization
    combined_rendering_data = [point for segment in aligned_rendering_data for point in segment]
    combined_experiment_data = [point for segment in aligned_experiment_data for point in segment]
    combined_experiment_kinetic = [point for segment in aligned_experiment_kinetic for point in segment]
    combined_rendering_indices = [point for segment in aligned_rendering_indices for point in segment]
    combined_experiment_indices = [point for segment in aligned_experiment_indices for point in segment]

    kinetic_output_folder = os.path.join(output_path, "kinetic")
    os.makedirs(kinetic_output_folder, exist_ok=True)
    kinetic_output_file = os.path.join(kinetic_output_folder, f"kinetic_{folder_name}.txt")
    save_list_to_txt(kinetic_output_file, combined_experiment_kinetic)
    print(kinetic_output_file)
    
    # plt.figure(figsize=(10, 5))
    # plt.plot(experiment_normalized_data[:500,0], label='experiment-0')
    # plt.scatter(experiment_peak_indices, experiment_peak_points, color='r', zorder=5, label="Peak Points")
    # # plt.plot(rendering_normalized_data[:,0], label='rendering-0')
    # plt.ylabel('Value')
    # plt.legend(loc='upper left', bbox_to_anchor=(-0.2, 1))
    # plt.title('Original experiment data')
    # plt.show()

    # plt.figure(figsize=(10, 5))
    # plt.plot(rendering_normalized_data[:,0], label='rendering-0')
    # plt.scatter(rendering_peak_indices, rendering_peak_points, color='r', zorder=5, label="Peak Points")
    # plt.ylabel('Value')
    # plt.legend(loc='upper left', bbox_to_anchor=(-0.2, 1))
    # plt.title('Original rendering data')
    # plt.show() 
    # print(combined_rendering_indices)
    # print(combined_experiment_indices)
    # print(len(combined_rendering_data), len(combined_experiment_data))
    plt.figure(figsize=(10, 5))
    plt.plot(combined_rendering_data, label='rendering')
    plt.plot(combined_experiment_data, label='experiment')
    plt.ylabel('Value')
    plt.legend(loc='upper left', bbox_to_anchor=(-0.2, 1))
    plt.title('Aligned data')
    plt.show()


############################### Pired Data Combination ############################### 
    if len(combined_rendering_indices) != len(combined_experiment_indices):
        print(f"Error: Length mismatch! selected rendering data has {len(combined_rendering_indices)} elements, but selected experiment data has {len(combined_experiment_indices)} elements.")
        break
    else:
        idx = 0
        for rendering_idx, experiment_idx in zip(combined_rendering_indices, combined_experiment_indices):
            output_folder = os.path.join(output_path, folder_name)
            os.makedirs(output_folder, exist_ok=True)
            output_file = os.path.join(output_folder, f"Paired_{idx+1}.png")
            experiment_image_file = os.path.join(experiment_image_path, folder_name, f"ANewOT{int(experiment_idx+2):05d}.jpg")
            rendering_image_file = os.path.join(rendering_image_path, rendering_folder_name, f"Image_{rendering_idx+1}.png")
            # print(output_file, experiment_image_file, rendering_image_file)

            rendering_image = Image.open(rendering_image_file).convert("RGB")
            rendering_image = center_crop(rendering_image)
            experiment_image = Image.open(experiment_image_file).convert("RGB")
            # print(rendering_image.size, experiment_image.size)
            
            rendering_image = rendering_image.resize((256, 256))
            experiment_image = experiment_image.resize((256, 256))
            # print(rendering_image.size, experiment_image.size)


            combined_image = Image.new("RGB", (512, 256))
            combined_image.paste(rendering_image, (0, 0))  # Left side
            combined_image.paste(experiment_image, (256, 0))  # Right side
            # print(combined_image.size)

            # plt.figure(figsize=(6, 6))  # Set the figure size
            # plt.imshow(combined_image)
            # plt.axis("off")  # Turn off the axis
            # plt.title(f"Combined Image: {idx}")
            # plt.show()

            # Save the combined image
            combined_image.save(output_file)
            # print(f"Saved combined image to {output_file}")

            idx += 1

In [ ]:
############################### Draw ###################################
# Plot Rendering
plt.figure(figsize=(6, 4))
x_coords = [0,   20, 60, 100, 140, 180, 220, 250]
y_coords = [0.5, 1,   0,   1,   0,   1,   0, 0.75]
x_interpolated = np.linspace(min(x_coords), max(x_coords), num=100)  # Higher resolution
y_interpolated = np.interp(x_interpolated, x_coords, y_coords)
plt.plot(rendering_normalized_data[:250, 0], label='Rendering: Laplacian of Gaussian', c='palevioletred')
plt.plot(x_interpolated, y_interpolated, label='Rendering: Normalised Depth', c='orange', linestyle='--')
plt.ylabel('Value')
plt.xlabel('Image Idx')
plt.legend(loc='center left')
plt.title('Rendering Image Features')
plt.savefig("Rendering.png")
plt.show()

# Plot Experiment
plt.figure(figsize=(6, 4))
plt.plot(experiment_normalized_data[:250,0], label='Experiment: Laplacian of Gaussian', c='royalblue')
plt.plot(experiment_normalized_data[:250,1], label='Experiment: Normalised Depth', color='orange', linestyle='--')
# plt.scatter(experiment_peak_indices, experiment_peak_points, color='r', zorder=5, label="Peak Points")
plt.ylabel('Value')
plt.xlabel('Image Idx')
plt.legend(loc='center left') #, bbox_to_anchor=(-0.2, 1)
plt.title('Experiment Image Features')
plt.savefig("Experiment.png")
plt.show()

# Plot Aligned
plt.figure(figsize=(6, 4))
plt.plot(combined_rendering_data[:250], label='Rendering: Laplacian of Gaussian', c='palevioletred')
plt.plot(combined_experiment_data[:250], label='Experiment: Laplacian of Gaussian', c='royalblue')
plt.ylabel('Value')
plt.xlabel('Image Idx')
plt.legend(loc='center left')
plt.title('Aligned Feature')
plt.savefig("Aligned.png")
plt.show()
############################### Draw ###################################

In [ ]:
#### Split ALL paired images into Train/Val/Test for PixGAN Training

import os
import shutil
import random
source_folder = "./xxx/Pix2Pix_All"
train_output_folder = "./xxx/Gen_Model_All/train"
val_output_folder   = "./xxx/Gen_Model_All/val"
test_output_folder  = "./xxx/Gen_Model_All/Test"
test_kinematic_output_folder = "./xxx/Gen_Model_All"

os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(val_output_folder, exist_ok=True)
os.makedirs(test_output_folder, exist_ok=True)

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

subfolders = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]

train_idx = 1
val_idx = 1
test_idx = 1

test_kinematic_dst = os.path.join(test_kinematic_output_folder, "test_kinematic.txt")  
aligned_kinematic = []

pose_segment = []
pose_segment_idx = 0
pose_segment_dst = os.path.join(test_kinematic_output_folder, "pose_segment.txt")

for subfolder in subfolders:
    # print("***: ", subfolder)
    if subfolder == '.ipynb_checkpoints':
        # print(subfolder)
        continue
    
    subfolder_path = os.path.join(source_folder, subfolder)
    kinematic_file_path = os.path.join(source_folder+"_kinetic", "kinetic_"+ subfolder + '.txt') #kinetic/kinetic_P60_R70.txt
    kinematic_values = np.loadtxt(kinematic_file_path)

    files = os.listdir(subfolder_path)
    print(kinematic_file_path)
    if len(files) != len(kinematic_values):
        print(f"wrong! len(files): {len(files)} != len(kinematic_values): {len(kinematic_values)}")
        break
    
    combined = list(zip(files, kinematic_values))
    random.shuffle(combined)
    files, kinematic_values = zip(*combined)
    
    total_files = len(files)
    train_count = int(total_files * train_ratio)
    val_count = int(total_files * val_ratio)
    
    train_files = files[:train_count]
    val_files = files[train_count:train_count + val_count]
    test_files = files[train_count + val_count:]

    train_kinematic = kinematic_values[:train_count]
    val_kinematic = kinematic_values[train_count:train_count + val_count]
    test_kinematic = kinematic_values[train_count + val_count:]
    
    # Save Train files
    for file, kinematic in zip(train_files, train_kinematic):
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(train_output_folder, f"Paired_{train_idx}.png") 
        # print(dst)
        shutil.copy(src, dst)
        train_idx += 1

    # Save Val files
    for file, kinematic in zip(val_files, val_kinematic):
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(val_output_folder, f"Paired_{val_idx}.png") 
        shutil.copy(src, dst)
        # print(dst)
        val_idx += 1
    
    # Save Test files
    for file, kinematic in zip(test_files, test_kinematic):
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(test_output_folder, f"Paired_{test_idx}.png") 
        shutil.copy(src, dst)
        # print(dst)
        test_idx += 1

    aligned_kinematic.append(test_kinematic)
    pose_segment_idx += len(test_files)
    pose_segment.append(pose_segment_idx)
    print(f"Processed subfolder: {subfolder}")
    print(f"Train: {len(train_files)}/{train_idx} files, Val: {len(val_files)}/{val_idx} files, Test: {len(test_files)}/{test_idx} files")

combined_kinematic = [point for segment in aligned_kinematic for point in segment]    
print(len(combined_kinematic))
save_list_to_txt(test_kinematic_dst, combined_kinematic)
save_list_to_txt(pose_segment_dst, pose_segment)
print("Data split and saved successfully.")


In [ ]:
#### Split Part paired images into Train/Val/Test for PixGAN Training

import os
import shutil
import random
source_folder = "./xxx/Pix2Pix_All"
train_output_folder = "./xxx/Gen_Model_Part/train"
val_output_folder   = "./xxx/Gen_Model_Part/val"
test_output_folder  = "./xxx/Gen_Model_Part/Test"
test_kinematic_output_folder = "./xxx/Gen_Model_Part"

os.makedirs(train_output_folder, exist_ok=True)
os.makedirs(val_output_folder, exist_ok=True)
os.makedirs(test_output_folder, exist_ok=True)

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

subfolders = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]

train_idx = 1
val_idx = 1
test_idx = 1

test_kinematic_dst = os.path.join(test_kinematic_output_folder, "test_kinematic.txt")  
aligned_kinematic = []

pose_segment = []
pose_segment_idx = 0
pose_segment_dst = os.path.join(test_kinematic_output_folder, "pose_segment.txt")

for subfolder in subfolders:
    # print("***: ", subfolder)
    if subfolder == '.ipynb_checkpoints':
        # print(subfolder)
        continue
    elif subfolder == 'P0_R20':
        # print(subfolder)
        continue
    elif subfolder == 'P10_R30':
        # print(subfolder)
        continue
    elif subfolder == 'P20_R40':
        # print(subfolder)
        continue
    elif subfolder == 'P30_R50':
        # print(subfolder)
        continue
    elif subfolder == 'P40_R60':
        # print(subfolder)
        continue
    
    subfolder_path = os.path.join(source_folder, subfolder)
    kinematic_file_path = os.path.join(source_folder+"_kinetic", "kinetic_"+ subfolder + '.txt') #kinetic/kinetic_P60_R70.txt
    kinematic_values = np.loadtxt(kinematic_file_path)

    files = os.listdir(subfolder_path)
    print(kinematic_file_path)
    if len(files) != len(kinematic_values):
        print(f"wrong! len(files): {len(files)} != len(kinematic_values): {len(kinematic_values)}")
        break
    
    combined = list(zip(files, kinematic_values))
    random.shuffle(combined)
    files, kinematic_values = zip(*combined)
    
    total_files = len(files)
    train_count = int(total_files * train_ratio)
    val_count = int(total_files * val_ratio)
    
    train_files = files[:train_count]
    val_files = files[train_count:train_count + val_count]
    test_files = files[train_count + val_count:]

    train_kinematic = kinematic_values[:train_count]
    val_kinematic = kinematic_values[train_count:train_count + val_count]
    test_kinematic = kinematic_values[train_count + val_count:]
    
    # Save Train files
    for file, kinematic in zip(train_files, train_kinematic):
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(train_output_folder, f"Paired_{train_idx}.png") 
        # print(dst)
        shutil.copy(src, dst)
        train_idx += 1

    # Save Val files
    for file, kinematic in zip(val_files, val_kinematic):
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(val_output_folder, f"Paired_{val_idx}.png") 
        shutil.copy(src, dst)
        # print(dst)
        val_idx += 1
    
    # Save Test files
    for file, kinematic in zip(test_files, test_kinematic):
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(test_output_folder, f"Paired_{test_idx}.png") 
        shutil.copy(src, dst)
        # print(dst)
        test_idx += 1

    aligned_kinematic.append(test_kinematic)
    pose_segment_idx += len(test_files)
    pose_segment.append(pose_segment_idx)
    print(f"Processed subfolder: {subfolder}")
    print(f"Train: {len(train_files)}/{train_idx} files, Val: {len(val_files)}/{val_idx} files, Test: {len(test_files)}/{test_idx} files")

combined_kinematic = [point for segment in aligned_kinematic for point in segment]    
print(len(combined_kinematic))
save_list_to_txt(test_kinematic_dst, combined_kinematic)
save_list_to_txt(pose_segment_dst, pose_segment)
print("Data split and saved successfully.")


In [ ]:
### Separate the test image (of GAN Model Traning) into different pose for Pose Model Traning 
### Both experiment & generated - ALL/PART
#  ./xxx_results_RAL/xxx_PixGAN_All_Data/Test_latest/images
#  ./xxx_results_RAL/xxx_PixGAN_Part_Data/Test_latest/images

import os
import shutil
import random
name_folder = "./xxx/Pix2Pix_All"
# source_folder = "./pytorch-CycleGAN-and-pix2pix/results/pix2pix_unet_256_lsgan/Test_20/images"
# source_folder = "./xxx_results_RAL/xxx_PixGAN_All_Data/Test_latest/images"
pose_segment_path = "./xxx/Gen_Model_All/pose_segment.txt"
experiment_output_path = "./xxx/Pose_Model/Experiment_All"
# generated_output_path = "./xxx/Pose_Model/Generated_All"

source_folder = "./xxx_results_RAL/xxx_PixGAN_Part_Data/Test_latest/images"
# pose_segment_path = "./xxx/Gen_Model_Part/pose_segment.txt"
# experiment_output_path = "./xxx/Pose_Model/Experiment_Part"
generated_output_path = "./xxx/Pose_Model/Generated_Part"
os.makedirs(experiment_output_path, exist_ok=True)
os.makedirs(generated_output_path,  exist_ok=True)

def read_int_from_txt(filename):
    with open(filename, 'r') as f:
        data_scores = [int(line.strip()) for line in f]
    return data_scores

pose_folder_value = read_int_from_txt(pose_segment_path)
pose_folder = [0] + pose_folder_value

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

subfolders = [f for f in os.listdir(name_folder) if os.path.isdir(os.path.join(name_folder, f))]
all_files = os.listdir(source_folder)
files = [file for file in all_files if file.endswith('fake_B.png')]


for i in range(len(pose_folder) - 1):
    start_idx = pose_folder[i]
    end_idx = pose_folder[i + 1]
    idx = start_idx+1
    # print(start_idx, end_idx)
    pose_name = subfolders[i]
    
    save_idx = 1
    pose_file = files[start_idx:end_idx]
    pose_file_len = len(pose_file)

    idx = start_idx+1
    experiment_dst_folder = os.path.join(experiment_output_path, pose_name)
    generated_dst_folder = os.path.join(generated_output_path, pose_name)
    os.makedirs(experiment_dst_folder, exist_ok=True)
    os.makedirs(generated_dst_folder, exist_ok=True)
    for file in pose_file: #[:pose_file_len//2]: # save only half of the images
        experiment_src = os.path.join(source_folder, f"Paired_{idx}_real_B.png")
        generated_src  = os.path.join(source_folder, f"Paired_{idx}_fake_B.png")
        experiment_dst = os.path.join(experiment_dst_folder, pose_name + f"_Image_{save_idx}.png") 
        generated_dst  = os.path.join(generated_dst_folder, pose_name + f"_Image_{save_idx}.png") 
        # print(experiment_src)
        # print(generated_src)
        # print(experiment_dst)
        # print(generated_dst)
        shutil.copy(experiment_src, experiment_dst)
        shutil.copy(generated_src, generated_dst)
        # print(idx)
        
        save_idx += 1
        idx += 1
#     print(save_idx, pose_file_len)
print(idx)
print("All generated test data is separated into pose files")


In [ ]:
### Separate test image into train/val/test for pose estimation
# xxx_GAN/xxx/Pose_Model/Experiment_All
# xxx_GAN/xxx/Pose_Model/Generated_All
# xxx_GAN/xxx/Pose_Model/Generated_Part

import os
import shutil
import random
name_folder = "./xxx/Pix2Pix_All"

# source_folder = "./xxx/Pose_Model/Experiment_All"
# train_output_folder = "./xxx/Pose_Model/Experiment_All_separate/Train"
# val_output_folder   = "./xxx/Pose_Model/Experiment_All_separate/Val"
# test_output_folder  = "./xxx/Pose_Model/Experiment_All_separate/Test"

# source_folder = "./xxx/Pose_Model/Generated_All"
# train_output_folder = "./xxx/Pose_Model/Generated_All_separate/Train"
# # val_output_folder   = "./xxx/Pose_Model/Generated_All_separate/Val"
# # test_output_folder  = "./xxx/Pose_Model/Generated_All_separate/Test"

source_folder = "./xxx/Pose_Model/Generated_Part"
train_output_folder = "./xxx/Pose_Model/Generated_Part_separate/Train"
# val_output_folder   = "./xxx/Pose_Model/Generated_Part_separate/Val"
# test_output_folder  = "./xxx/Pose_Model/Generated_Part_separate/Test"

os.makedirs(train_output_folder, exist_ok=True)
# os.makedirs(val_output_folder, exist_ok=True)
# os.makedirs(test_output_folder, exist_ok=True)

# train_ratio = 0.84
# val_ratio = 0.01
# test_ratio = 0.15

subfolders = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(name_folder, f))]

train_idx = 1
# val_idx = 1
# test_idx = 1

for subfolder in subfolders:
    subfolder_path = os.path.join(source_folder, subfolder)
    files = os.listdir(subfolder_path)
    random.shuffle(files)
    
    total_files = len(files)
    # train_count = int(total_files * train_ratio)
    # val_count = int(total_files * val_ratio)
    
    train_files = files
    # train_files = files[:train_count]
    # val_files = files[train_count:train_count + val_count]
    # test_files = files[train_count + val_count:]
    
    # Save Train files
    for file in train_files:
        if file == '.ipynb_checkpoints':
                # print(subfolder)
                continue
        src = os.path.join(subfolder_path, file)
        dst = os.path.join(train_output_folder, file) 
        # print(src)
        shutil.copy(src, dst)
        train_idx += 1

    # Save Val files
    # for file in val_files:
    #     if file == '.ipynb_checkpoints':
    #             # print(subfolder)
    #             continue
    #     src = os.path.join(subfolder_path, file)
    #     dst = os.path.join(val_output_folder, file) 
    #     shutil.copy(src, dst)
    #     # print(src)
    #     val_idx += 1
    
    # Save Test files
    # for file in test_files:
    #     if file == '.ipynb_checkpoints':
    #             # print(subfolder)
    #             continue
    #     src = os.path.join(subfolder_path, file)
    #     dst = os.path.join(test_output_folder, file) 
    #     shutil.copy(src, dst)
    #     # print(dst)
    #     test_idx += 1

    print(f"Processed subfolder: {subfolder}")
    print(f"Train: {len(train_files)}/{train_idx} files") #, Val: {len(val_files)}/{val_idx} files, Test: {len(test_files)}/{test_idx} files")

print("Data split and saved successfully.")
